In [27]:
import pandas as pd
import statsmodels.api as sm

In [28]:
SP = pd.read_csv('MSCI.csv', parse_dates=True, index_col='Date',)
SP

,Price MSCI,Price GN,Price Demant,Price Sonova
Date,,,,
2023-01-04,"2.829,20","152,8","260,4","282,5"
2023-01-03,"2.791,44","153,9","240,2","268,6"
2023-01-02,"2.714,57","152,4","210,9","231,2"
2023-01-01,"2.785,00","167,8","192,85","227,9"
2022-01-12,"2.602,69","159,75","192,55","219,3"
...,...,...,...,...
2018-01-08,"2.175,50","333,1","261,2","182,68"
2018-01-07,"2.153,10",304,"304,6","181,59"
2018-01-06,"2.089,30","290,8","256,8","176,67"


In [29]:
SP = SP[SP.columns[SP.columns.isin(['Date','Price GN', 'Price MSCI'])]]
SP['Price GN'] = SP['Price GN'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
SP['Price MSCI'] = SP['Price MSCI'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
SP

/tmp/ipykernel_2856/1274191198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['Price GN'] = SP['Price GN'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
/tmp/ipykernel_2856/1274191198.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['Price MSCI'] = SP['Price MSCI'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)


,Price MSCI,Price GN
Date,,
2023-01-04,2829.20,152.80
2023-01-03,2791.44,153.90
2023-01-02,2714.57,152.40
2023-01-01,2785.00,167.80
2022-01-12,2602.69,159.75
...,...,...
2018-01-08,2175.50,333.10
2018-01-07,2153.10,304.00
2018-01-06,2089.30,290.80


In [30]:
SP.sort_values(by='Date', inplace = True) 
SP

/tmp/ipykernel_2856/1494837875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP.sort_values(by='Date', inplace = True)


,Price MSCI,Price GN
Date,,
2018-01-04,2086.51,217.40
2018-01-05,2092.92,243.10
2018-01-06,2089.30,290.80
2018-01-07,2153.10,304.00
2018-01-08,2175.50,333.10
...,...,...
2022-01-12,2602.69,159.75
2023-01-01,2785.00,167.80
2023-01-02,2714.57,152.40


In [36]:
daily_ret = SP.pct_change(1)
daily_ret_clean = daily_ret.dropna(axis=0)
daily_ret_clean.head()

,Price MSCI,Price GN
Date,,
2018-01-05,0.003072,0.118215
2018-01-06,-0.001730,0.196216
2018-01-07,0.030537,0.045392
2018-01-08,0.010404,0.095724
2018-01-09,0.003912,-0.060642


In [37]:

# split dependent and independent variable
X = daily_ret_clean['Price MSCI']
y = daily_ret_clean['Price GN']

# Add a constant to the independent value
X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y, X1)

# fit model and print results
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Price GN   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     24.15
Date:                Wed, 19 Apr 2023   Prob (F-statistic):           7.66e-06
Time:                        12:10:19   Log-Likelihood:                 58.963
No. Observations:                  60   AIC:                            -113.9
Df Residuals:                      58   BIC:                            -109.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0069      0.012     -0.580      0.5

In [38]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)

print(f'Beta = {slope}')

Beta = 1.125717224222587
